In [2]:
%pip list

Package            Version
------------------ -----------
asttokens          3.0.0
comm               0.2.2
customtkinter      5.2.2
darkdetect         0.8.0
debugpy            1.8.12
decorator          5.1.1
executing          2.2.0
ipykernel          6.29.5
ipython            8.32.0
jedi               0.19.2
jupyter_client     8.6.3
jupyter_core       5.7.2
matplotlib-inline  0.1.7
nest-asyncio       1.6.0
numpy              2.2.2
opencv-python      4.11.0.86
packaging          24.2
parso              0.8.4
pexpect            4.9.0
pillow             11.1.0
pip                24.3.1
platformdirs       4.3.6
prompt_toolkit     3.0.50
psutil             6.1.1
ptyprocess         0.7.0
pure_eval          0.2.3
Pygments           2.19.1
python-dateutil    2.9.0.post0
pyzmq              26.2.1
realesrgan-ncnn-py 2.0.0
setuptools         75.6.0
six                1.17.0
stack-data         0.6.3
tornado            6.4.2
traitlets          5.14.3
typing_extensions  4.12.2
wcwidth            0

In [3]:
!python -V

Python 3.10.5


In [5]:
import customtkinter as ctk
from tkinter import filedialog

# 外観モードとテーマの設定（任意）
ctk.set_appearance_mode("light")    # "light"または"dark"
ctk.set_default_color_theme("blue")   # カラーテーマ

# メインウィンドウの作成
app = ctk.CTk()
app.geometry("400x200")
app.title("ディレクトリ選択のデモ")

def select_directory():
    # ディレクトリ選択ダイアログを表示
    directory_path = filedialog.askdirectory(title="ディレクトリを選択してください")
    if directory_path:
        # 選択されたパスをラベルに表示
        label.configure(text=f"選択されたディレクトリ:\n{directory_path}")
    else:
        label.configure(text="ディレクトリが選択されませんでした")

# ボタンウィジェット：クリックでディレクトリ選択ダイアログを表示
select_button = ctk.CTkButton(app, text="ディレクトリを選択", command=select_directory)
select_button.pack(pady=20)

# 選択結果を表示するラベル
label = ctk.CTkLabel(app, text="まだディレクトリが選択されていません")
label.pack(pady=20)

# メインループ開始
app.mainloop()


In [ ]:
! .\realesrgan-ncnn-vulkan.exe -i .\assets\input.jpg -o .\assets\output.jpg -n realesrgan-x4plus -s 2 -t 32

[0 NVIDIA GeForce RTX 4060 Ti]  queueC=2[8]  queueG=0[16]  queueT=1[2]
[0 NVIDIA GeForce RTX 4060 Ti]  bugsbn1=0  bugbilz=205  bugcopc=0  bugihfa=0
[0 NVIDIA GeForce RTX 4060 Ti]  fp16-p/s/a=1/1/1  int8-p/s/a=1/1/1
[0 NVIDIA GeForce RTX 4060 Ti]  subgroup=32  basic=1  vote=1  ballot=1  shuffle=1
0.00%
0.08%
0.16%
0.24%
0.31%
0.39%
0.47%
0.55%
0.63%
0.71%
0.79%
0.87%
0.94%
1.02%
1.10%
1.18%
1.26%
1.34%
1.42%
1.49%
1.57%
1.65%
1.73%
1.81%
1.89%
1.97%
2.05%
2.12%
2.20%
2.28%
2.36%
2.44%
2.52%
2.60%
2.68%
2.75%
2.83%
2.91%
2.99%
3.07%
3.15%
3.23%
3.30%
3.38%
3.46%
3.54%
3.62%
3.70%
3.78%
3.86%
3.93%
4.01%
4.09%
4.17%
4.25%
4.33%
4.41%
4.48%
4.56%
4.64%
4.72%
4.80%
4.88%
4.96%
5.04%
5.11%
5.19%
5.27%
5.35%
5.43%
5.51%
5.59%
5.66%
5.74%
5.82%
5.90%
5.98%
6.06%
6.14%
6.22%
6.29%
6.37%
6.45%
6.53%
6.61%
6.69%
6.77%
6.85%
6.92%
7.00%
7.08%
7.16%
7.24%
7.32%
7.40%
7.47%
7.55%
7.63%
7.71%
7.79%
7.87%
7.95%
8.03%
8.10%
8.18%
8.26%
8.34%
8.42%
8.50%
8.58%
8.65%
8.73%
8.81%
8.89%
8.97%
9.05%
9.13%
9

In [9]:
import subprocess

def run_realesrgan(input_path, output_path, model_name="realesrgan-x4plus", scale=2, tile_size=32):
    command = [
        ".\\realesrgan-ncnn-vulkan_local_build.exe",
        "-i", input_path,
        "-o", output_path,
        "-n", model_name,
        "-s", str(scale),
        "-t", str(tile_size)
    ]

    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        print("Execution completed successfully.")
        print("Output:\n", result.stdout)
    except subprocess.CalledProcessError as e:
        print("Error occurred while running the command.")
        print("Error details:\n", e.stderr)

if __name__ == '__main__':
    input_image = ".\\assets\\input.jpg"
    output_image = ".\\assets\\output.jpg"
    run_realesrgan(input_image, output_image)


: 

In [8]:
import subprocess

def get_gpu_info():
    try:
        # GPUのインデックスと名前を取得
        result = subprocess.run(['nvidia-smi', '--query-gpu=index,name', '--format=csv,noheader'],
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        # 各行を解析してリストに格納
        gpu_info = [line.strip().split(', ') for line in result.stdout.strip().split('\n')]
        return [{"id": int(info[0]), "name": info[1]} for info in gpu_info]
    except Exception as e:
        print(f"Error getting GPU info: {e}")
        return []

if __name__ == '__main__':
    gpus = get_gpu_info()
    for gpu in gpus:
        print(f"GPU ID: {gpu['id']}, Name: {gpu['name']}")


GPU ID: 0, Name: NVIDIA GeForce RTX 4060 Ti


In [3]:
from realesrgan_ncnn_py import Realesrgan
from PIL import Image

input_image = ".\\assets\\input.jpg"
output_image = ".\\assets\\output.jpg"

realesrgan = Realesrgan(gpuid=0, tilesize=0, model=4)  # GPU IDを指定
with Image.open(input_image) as img:
    image = realesrgan.process_pil(img)
    image.save(output_image)

ModuleNotFoundError: No module named 'realesrgan_ncnn_vulkan_wrapper'

In [ ]:
import subprocess

def run_realesrgan(input_path, output_path, model_name="realesrgan-x4plus", scale=2, tile_size=32):
    command = [
        ".\\realesrgan-ncnn-vulkan.exe",
        "-i", input_path,
        "-o", output_path,
        "-n", model_name,
        "-s", str(scale),
        "-t", str(tile_size)
    ]

    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        print("Execution completed successfully.")
        print("Output:\n", result.stdout)
    except subprocess.CalledProcessError as e:
        print("Error occurred while running the command.")
        print("Error details:\n", e.stderr)

if __name__ == '__main__':
    input_image = ".\\assets\\input.jpg"
    output_image = ".\\assets\\output.jpg"
    run_realesrgan(input_image, output_image)


In [1]:
!c:\users\ryosh\appdata\local\pip\cache\wheels\85\70\21\31cbffb7c34454d900d2a799e69c7732fbf1c1adc9e927c540\python_realesrgan_ncnn_vulkan-1.1-cp310-cp310-win_amd64.whl

In [4]:
from PIL import Image
from realesrgan_ncnn_vulkan import RealESRGAN

# モデルのロード
model = RealESRGAN()
model.load("./models/realesrgan-x4plus.param", "./models/realesrgan-x4plus.bin")
model.scale = 4

# 入力画像の読み込み
input_image = Image.open(".\\assets\\input.jpg")

# 画像の処理
output_bytes = model.process(
    input_image.width,
    input_image.height,
    input_image.tobytes(),
    3  # RGBモード
)

# 出力画像の保存
output_image = Image.frombytes(
    "RGB",
    (input_image.width * model.scale, input_image.height * model.scale),
    output_bytes
)
output_image.save(".\\assets\\output.jpg")
print("Upscaling completed successfully.")


ModuleNotFoundError: No module named 'realesrgan_ncnn_vulkan'